In [16]:
def AND_gate(x1, x2):
  w1 = 0.5
  w2 = 0.5
  b = -0.7
  result = x1*w1 + x2*w2 + b
  if result <= 0:
    return 0
  else:
    return 1

def NAND_gate(x1, x2):
    w1=-0.5
    w2=-0.5
    b=0.7
    result = x1*w1 + x2*w2 + b
    if result <= 0:
        return 0
    else:
        return 1

    
#  XOR 게이트 구현
def XOR_gate(x1, x2):
    w1=0.6
    w2=0.6
    b=-0.5
    result = x1*w1 + x2*w2 + b
    if result <= 0:
      return AND_gate(x1, x2)  # 층을 하나 더 추가함으로 XOR 게이트를 구현 가능
    else:
      return NAND_gate(x1, x2)

# 두개의 입력값이 서로 다른 값을 갖고 있을 때만 출력값이 1이 되고 나머진 0이 됨
XOR_gate(0, 0), XOR_gate(0, 1), XOR_gate(1, 0), XOR_gate(1, 1)

(0, 1, 1, 0)

In [26]:
# XOR 게이트 구현(PyTorch)
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(1)

# XOR 문제를 풀기 위한 입력과 출력 정의
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# 다층 퍼셉트론 설계. 입력층, 은닉층1, 은닉층2, 은닉층3, 출력층을 가지는 인공신경망
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
    nn.Sigmoid(),
    nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
    nn.Sigmoid()
).to(device)

# 비용 함수와 옵티마이저 선언
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

# 훈련 수행. 각 에포크마다 역전파 수행
for epoch in range(1001):
    optimizer.zero_grad() # backward()를 호출하고 나면 각 파라미터들의 .grad 값에 변화도가 저장이 된다. 저장된 값이 다음 루프의 업데이트에도 간섭하지 않도록 .grad 값들을 0으로 초기화 시켜준다.
    # **zero_grad() 인자로 넘겨주는 set_to_none=False는 True로 넘기면 메모리 사용량이 적어져서 퍼포먼스가 소폭 향상 되지만, manual하게 .grad 값에 접근해서 수정을 할 때 다르게 작동해서 쓰이지 않는다. 
    
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.7319304943084717
100 0.693137526512146
200 0.6931369304656982
300 0.6931360960006714
400 0.6931353807449341
500 0.6931347250938416
600 0.6931338310241699
700 0.6931329965591431
800 0.6931321620941162
900 0.6931310892105103
1000 0.6931300759315491


In [28]:
# 10001
for epoch in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.6931300759315491
100 0.6931289434432983
200 0.6931277513504028
300 0.6931264400482178
400 0.6931250691413879
500 0.6931235790252686
600 0.6931218504905701
700 0.6931201219558716
800 0.6931182146072388
900 0.6931160688400269
1000 0.6931138038635254
1100 0.6931111812591553
1200 0.693108320236206
1300 0.6931052207946777
1400 0.6931016445159912
1500 0.6930976510047913
1600 0.6930931806564331
1700 0.6930881142616272
1800 0.693082332611084
1900 0.6930756568908691
2000 0.6930679678916931
2100 0.6930590867996216
2200 0.6930485963821411
2300 0.6930360794067383
2400 0.6930210590362549
2500 0.6930028796195984
2600 0.6929803490638733
2700 0.6929522156715393
2800 0.6929161548614502
2900 0.6928691267967224
3000 0.6928060054779053
3100 0.6927184462547302
3200 0.6925923228263855
3300 0.6924009323120117
3400 0.6920909881591797
3500 0.6915416121482849
3600 0.6904304623603821
3700 0.6876454949378967
3800 0.6771503686904907
3900 0.59931880235672
4000 0.4278465509414673
4100 0.02223077416419983
4200 0.

In [29]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[9.6978256e-05]
 [9.9989307e-01]
 [9.9986112e-01]
 [1.3367381e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
